In [139]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from nltk import NaiveBayesClassifier
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
import re
from sklearn import svm
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier

In [45]:
training_data = pd.read_csv('C:\\Users\\KSHITIJ AGGARWAL\\Desktop\\CN PDFs\\ML DS content\\Project - Twitter Sentiment Analysis\\training_twitter_x_y_train.csv', delimiter = ',')
testing_data = pd.read_csv('C:\\Users\\KSHITIJ AGGARWAL\\Desktop\\CN PDFs\\ML DS content\\Project - Twitter Sentiment Analysis\\test_twitter_x_test.csv', delimiter = ',')

In [46]:
training_data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [47]:
training_data.count()

tweet_id                  10980
airline_sentiment         10980
airline                   10980
airline_sentiment_gold       31
name                      10980
negativereason_gold          24
retweet_count             10980
text                      10980
tweet_coord                 776
tweet_created             10980
tweet_location             7430
user_timezone              7403
dtype: int64

In [48]:
#cleaning data
training_data.drop(['tweet_id', 'airline_sentiment_gold', 'negativereason_gold', 'tweet_coord', 'tweet_created'], inplace = True, axis = 1)
training_data.head()

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,negative,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,positive,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,positive,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,Illinois,Central Time (US & Canada)
3,negative,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,Atlantic Time (Canada)
4,negative,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,NaN,Eastern Time (US & Canada)


In [49]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  10980 non-null  object
 1   airline            10980 non-null  object
 2   name               10980 non-null  object
 3   retweet_count      10980 non-null  int64 
 4   text               10980 non-null  object
 5   tweet_location     7430 non-null   object
 6   user_timezone      7403 non-null   object
dtypes: int64(1), object(6)
memory usage: 600.6+ KB


In [50]:
lemmatizer=WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\KSHITIJ
[nltk_data]     AGGARWAL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
mood_count = training_data['airline_sentiment'].value_counts()
mood_count

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [55]:
le_clf = LabelEncoder()
training_data['airline_sentiment'] = le_clf.fit_transform(training_data['airline_sentiment'])
training_data.head()

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,2,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,2,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,Illinois,Central Time (US & Canada)
3,0,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,Atlantic Time (Canada)
4,0,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,NaN,Eastern Time (US & Canada)


In [64]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\KSHITIJ
[nltk_data]     AGGARWAL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [66]:
def convert(original_tweets):
    letters = re.sub("[^a-zA-Z]", " ",original_tweets)
    words = letters.lower().split()
    stops = set(stopwords.words('english'))
    useful_words = [w for w in words if not w in stops]
    return (" ".join(useful_words))

training_data['updated_tweets'] = training_data['text'].apply(lambda x: convert(x))
testing_data['updated_tweets'] = testing_data['text'].apply(lambda x: convert(x))

In [67]:
training_data_final_tweets = []
for t in training_data['updated_tweets']:
    training_data_final_tweets.append(t)

testing_data_final_tweets = []
for t in testing_data['updated_tweets']:
    testing_data_final_tweets.append(t)

In [69]:
cv_clf = CountVectorizer(analyzer = "word")
training_features = cv_clf.fit_transform(training_data_final_tweets)
testing_features = cv_clf.transform(testing_data_final_tweets)
training_features

<10980x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 107004 stored elements in Compressed Sparse Row format>

In [72]:
nb_clf = MultinomialNB()
nb_clf.fit(training_features, training_data['airline_sentiment'])
accuracy = accuracy_score(nb_clf.predict(training_features),training_data['airline_sentiment'])
print(accuracy)

0.8437158469945355


In [73]:
testing_features

<3660x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 33563 stored elements in Compressed Sparse Row format>

In [133]:
y_pred = nb_clf.predict(testing_features)
print(y_pred)
df = pd.DataFrame(y_pred)
for i in range(df.shape[0]):
    if df[0][i] == 0:
        df[0][i] = 'negative'
    elif df[0][i] == 1:
        df[0][i] = 'neutral'
    else:
        df[0][i] = 'positive'
df.head(20)

y_pred = np.array(df)
y_pred

[0 0 0 ... 1 2 1]


array([['negative'],
       ['negative'],
       ['negative'],
       ...,
       ['neutral'],
       ['positive'],
       ['neutral']], dtype=object)

In [136]:
rf_clf = ensemble.RandomForestClassifier()
rf_clf.fit(training_features, training_data['airline_sentiment'])
accuracy = accuracy_score(rf_clf.predict(training_features),training_data['airline_sentiment'])
print(accuracy)

0.995719489981785


In [131]:
y_pred = rf_clf.predict(testing_features)
y_pred
df = pd.DataFrame(y_pred)
for i in range(df.shape[0]):
    if df[0][i] == 0:
        df[0][i] = 'negative'
    elif df[0][i] == 1:
        df[0][i] = 'neutral'
    else:
        df[0][i] = 'positive'
df.head(20)

y_pred = np.array(df)
y_pred

array([['negative'],
       ['negative'],
       ['negative'],
       ...,
       ['neutral'],
       ['positive'],
       ['negative']], dtype=object)

In [138]:
svc_clf = svm.SVC()
svc_clf.fit(training_features, training_data['airline_sentiment'])
accuracy = accuracy_score(svc_clf.predict(training_features),training_data['airline_sentiment'])
print(accuracy)

0.920127504553734


In [140]:
y_pred = svc_clf.predict(testing_features)
y_pred
df = pd.DataFrame(y_pred)
for i in range(df.shape[0]):
    if df[0][i] == 0:
        df[0][i] = 'negative'
    elif df[0][i] == 1:
        df[0][i] = 'neutral'
    else:
        df[0][i] = 'positive'
df.head(20)

y_pred = np.array(df)
y_pred

array([['negative'],
       ['negative'],
       ['negative'],
       ...,
       ['neutral'],
       ['positive'],
       ['negative']], dtype=object)

In [141]:
np.savetxt("Twitter_sentiment_analysis_output.csv", y_pred, fmt = '%s', delimiter = ',')